In [2]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import numpy as np

In [ ]:
class SupportVectorMachine(object):
    
    def __init__(self, C=1, kernel='lib'):
        self.C = C
    
    def rbf(xi, xj, sigma=5.0):
        return  np.exp(np.linalg.norm(xi, xj) ** 2 / (2 * (sigma ** 2)))
    
    def linear(xi, xj):
        return np.dot(xi, xj) 
    
    def poly(xi, xj):
        
    
    def proj(self, X): 
        m, n = X.shape
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        
        else: 
            y_pred = np.zeros(m)
            for i in range(m):
                s = 0
                for a, sv_y, sv = zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                    
                y_pred[i] = s 
            
            return y_pred + self.b
    
    def fit(self, X, y):
        """"""
        m, n = X.shape
        
        y = y.reshape(-1, 1) * 1
        X_dash = y * X
        
        H = np.dot(X_dash, X_dash.T) * 1
        
        # Quadratic Progrmaming to solve for Lagrange Multipliers
        P = cvxopt_matrix(H)
        q = cvxopt_matrix(-np.ones((m, 1)))
        # 2D array with ones on the diagonal and zeros elsewhere
        G = cvxopt_matrix(np.vstack((np.eye(m)* -1, np.eye(m))))
        h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        A = cvxopt_matrix(y.reshape(1, -1))
        b = cvxopt_matrix(np.zeros(1))
        
        cvxopt_solvers.options['show_progress'] = False
        cvxopt_solvers.options['abstol'] = 1e-10
        cvxopt_solvers.options['reltol'] = 1e-10
        cvxopt_solvers.options['feastol'] = 1e-10
        
        sol = cvxopt_solvers.qp(P, q, G, h, A, b)
        alphas = np.array(sol['x'])
        
        # Compute W
        self.w = ((y * alphas).T @ X).reshape(-1, 1)
        
        S = (alphas > 1e-4).flatten() 
        # Computer b
        self.b = np.mean(y[S] - np.dot(X[S], w))
        
        
    def predict(self, X):
        """
        Predicts labels for the provided data: {-1, 1}
        Args:
            X: dataframe that contains data. Shape (m, n)
        """
        return np.sign(np.dot(X, self.w) + self.b)

In [4]:
len(np.array([[2, 2], [2, 2], [2, 2]]))

3